In [1]:
using Pickle, JSON, CSV, DataFrames, BenchmarkTools, Distributed, FileIO, Pickle, Unmarshal
using NetPricing
using JuMP
using Gurobi

In [ ]:
gamma = ...
gamma-1 = ...

# parameter
A, b, c
# variable
xg, λg, t, x



@constraint(model, gamma(A) * xg == gamma(b))
@constraint(model, gamma(A)' * λg <= gamma(c) + gamma(t))
@constraint(model, (c+t)' * x <= b' * gamma-1(λg))
@constraint(model, (gamma(c)+gamma(t))' * xg <= gamma(b)' * λg)



    prob = problem(primal)
    parentprob = parent(prob)

    a1 = tolled_arcs(prob)
    k = index(prob)
    Amap = arcmap(prob)

    x = primal.x[a1]
    tx = @variable(model, [a=a1], lower_bound = 0, base_name="tx[$k]")
    t = remap_t(model, prob)

    if isempty(tx)
        sumtx = 0.0
    else
        sumtx = sum(tx)
    end

    a1dict = Dict(a => i for (i, a) in enumerate(tolled_arcs(parentprob)))
    mapped_a1 = [a1dict[a] for a in Amap[a1]]
    M = @view M[mapped_a1]
    N = @view N[mapped_a1]

    # Linearization
    @constraint(model, tx .≤ M .* x)
    @constraint(model, t .- tx .≥ 0)
    @constraint(model, t .- tx .≤ N .* (1 .- x))

    c = cost_vector(prob)
    for ...
        xgo = gamma-1(xg)
        @constraint(model, c' * x + tx .≤ (c + t)' * xgo)
    
    # Run linearization-specific code
    linearize_commodity_extra(linearization, primal)

    return sumtx